В данном блокноте удаляю дубликаты session_id, причём те, у которых target_event = 0

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from warnings import filterwarnings
import matplotlib.pyplot as plt
plt.style.use('dark_background')

filterwarnings('ignore')

%store -r df

# для этого датафрейма 
#df = df.sample(frac=1).reset_index(drop=True)
#df = df.sample(n=50000).reset_index(drop=True)

res = df[~((df.duplicated(['session_id'])) & (df['target_event']==0) )]
# for col in res.columns:
#     res[col] = res[col].fillna('unknown')

In [ ]:
# def view_occupancy(df):
#     length = len(df)
#     dic = dict()
#     for col in df.columns:
#         percent_filled = (length-df[col].isna().sum())/length
#         dic[col] = percent_filled

#     dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
#     for k, v in dic:
#         print('{}: {:.4f}%'.format(k, v*100))

In [ ]:
# view_occupancy(res)

In [ ]:
# def foo(col_name):
#     df0 = res[res['target_event']==0].groupby([col_name]).agg({'target_event': 'count'}).rename(columns={'target_event': 'zeros'})
#     df1 = res[res['target_event']==1].groupby([col_name]).agg({'target_event': 'count'}).rename(columns={'target_event': 'ones'})
#     df_part = pd.merge(df0, df1, left_index=True, right_index=True, how='outer')
#     df_part['zeros'] = df_part['zeros'].fillna(0).astype(int)
#     df_part['ones'] = df_part['ones'].fillna(0).astype(int)
#     df_part['part'] = df_part.apply(lambda a: round(a['ones']/a['zeros'], 4), axis=1)
#     return df_part.drop(['zeros', 'ones'], axis=1)

## visit_date и visit_time
здесь делаю некоторые манипуляции с датой и временем: смотрю сколько было целевых действий за месяц и за час - оказывается, час и месяц влияют на целевое действие

In [15]:
# def visit_hour(x):
#     return h.loc[x]

# def visit_month(x):
#     return m.loc[x]

# res['visit_time'] = pd.to_datetime(res['visit_time'])
# res['hour'] = res['visit_time'].apply(lambda x: int(x.hour))

res['visit_date'] = pd.to_datetime(res['visit_date'])
res['month'] = res['visit_date'].apply(lambda x: int(x.month))

# df_plt = res.groupby(['month'])['target_event'].mean().to_frame().reset_index()
# df_plt.sort_values(by='target_event', ascending=False)

# res['mh'] = res.apply(lambda x: '{}-{}'.format(x['month'],x['hour']), axis=1)
# gr = res.groupby(['mh'])['target_event'].mean().to_frame().reset_index()
# gr.sort_values(by='target_event', ascending=False).head(10)

In [ ]:
# def foo_month(x):
#     try:
#         return gr.loc[gr['mh']==x]['target_event'].item()
#     except:
#         return 0
    

# res['mon'] = res['mh'].apply(lambda b: foo_month(b) )
# res = res.drop(['mh'], axis=1)

In [ ]:
# # res = res.drop(['month', 'hour'], axis=1)
# res[['mh', 'mon']].head(20)

In [ ]:
# h = foo('visit_hour')
# res['visit_hour'] = res['visit_hour'].apply(lambda h: visit_hour(h))

# m = foo('visit_month')
# res['visit_month'] = res['visit_month'].apply(lambda m: visit_month(m))

# # res = res.drop(['visit_date', 'visit_time'], axis=1)

In [ ]:
# res[res['target_event']==0]['visit_month'].value_counts()

## UTM_*
### utm_source

In [16]:
source_features = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs', 
                   'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm']
res['utm_source'] = res['utm_source'].fillna('0')

In [19]:
def foo_source(x):
    if x in source_features:
        return 2
    elif x == '0':
        return 0
    else:
        return 1
    return 0
# res['utm_source'] = res['utm_source'].apply(lambda x: 2 if x in source_features 1).astype(np.int8)
res['utm_source'] = res['utm_source'].apply(lambda x: foo_source(x))

### utm_keyword

In [4]:
res['utm_keyword'] = res.utm_keyword.fillna('0')
res['utm_keyword'] = res['utm_keyword'].apply(lambda a: 0 if a == '0' else 1).astype(np.int8)

### utm_medium

In [21]:
utm_medium_type = ['organic', 'referral', '(none)']

def foo_medium(x):
    if x in utm_medium_type:
        return 2
    elif x == '0':
        return 0
    else:
        return 1

# res['utm_medium_type'] = res['utm_medium'].apply(lambda x: 'organic' if x in ['organic', 'referral', '(none)'] else 'paid_traffic')
# res['utm_medium'] = res['utm_medium'].apply(lambda x: 2 if x in utm_medium_type else 1)
res['utm_medium'] = res['utm_medium'].apply(lambda x: foo_medium(x))

### utm_adcontent

In [ ]:
# def utm_adcontent(x):
#     return adcontent.loc[x]
# # 
# adcontent = foo('utm_adcontent')
# res['utm_adcontent'] = res['utm_adcontent'].apply(lambda y: utm_adcontent(y))

### utm_campaign

In [5]:
res['utm_campaign'] = res['utm_campaign'].fillna('0')

res['utm_campaign'] = res['utm_campaign'].apply(lambda a: 0 if a == '0' else 1).astype(np.int8)
# res = res.drop(['session_id', 'client_id', 'device_model', 'visit_number', 'utm_keyword'], axis=1)
# res.head()

## DEVICE_*

### device_category

device_category - 3 категории, пропусков нет. Из преобразований - будет последующее onehot-кодирование

### device_os, device_brand, device_browser

восстановим ОС из других значений device_*

In [ ]:
# best_brands = ['Samsung', 'Xiaomi', 'Huawei', 'POCO']

# dev_types = ['mobile', 'tablet']

# android_brands = ['Xiaomi', 'Samsung', 'Huawei', 'Vivo', 'LG',
#        'BQ', 'OnePlus', 'itel', 'Asus', 'Sony', 'Android', 'OPPO',
#        'Nokia', 'Realme', 'Alcatel', 'Infinix', 'Micromax', 'Blackview',
#        'Oukitel', 'ZTE', 'Motorola', 'HOMTOM', 'Google', 'Lenovo',
#        'China Phone', 'Neffos', 'Tecno', 'DOOGEE', 'Vsmart', 'Meizu', 'Umidigi', 'Leagoo', 'Highscreen', 'Ulefone', 'HTC',
#        'Cubot', 'Symphony', 'Vestel', 'Guophone', 'Wileyfox', 'Vertex',
#        'POCO', 'TCL', 'SonyEricsson', 'Nomu', 'Prestigio',
#        'Philips', 'LeEco', 'DEXP', 'Hisense', 'Vernee', 'Tele2', 'CAT',
#        'H96', 'Kyocera', 'Inoi', 'Acer', 'Kingplay', 'Sharp',
#        'MTC', 'iNew', 'Karbonn', 'Wiko', 'Gionee', 'Digma',
#        'Yota Devices', 'Kata', 'Poptel', 'Flylion', 'AGM', 'Teclast',
#        'Fly', 'MLS', 'LeTV', 'Nuu', 'Celkon', 'Chuwi', 'Walton',
#        'Haier', 'Coolpad', 'Xiaolajiao', 'Lava', 'General Mobile',
#        'Black Fox', 'Condor', 'Razer', 'Microsoft', 'Fujitsu',
#        'Alldocube', 'ZOJI', 'Amazon', 'PPTV', 'Jiake', 'Mito', 'Oysters',
#        'NOA', 'Caterpillar', 'SenseIT', 'Verizon', 'Komu', 'BLU', 'RCA',
#        'Cube', 'AT&T', 'Tonbux', 'Dark', 'Elephone', 'T-Mobile',
#        'Smartfren', 'Billion', 'Gome', 'A1', 'MTS', 'Geotel', 'BLUBOO',
#        'Ananda', 'MediaTek', 'KingSing', 'Ark', 'Essential',
#        'Dragon Touch', 'E&L', 'Fero',  'Wigor', 'Tanix',
#        'Mlais', 'InFocus', 'Smartisan', 'Beelink', 'Unihertz', 'Orbic',
#        'Honeywell', 'Archos', 'RED', 'Sonim', 'Jiayu', 'Evertek',
#        'Leegoog', 'QMobile', 'INUI', 'Artel', 'Irbis', 'China Mobile',
#        'Flexymove', 'Olla', 'Krip', 'Star', 'UMI', 'Iris', 'Ellipsis',
#        'LTC', 'Motive', 'Advan', 'M-HORSE', 'Winnovo', 'ThL', 'Nomi',
#        'Wings Mobile', 'MXQ', 'How', 'Centric', 'Maze', 'Corn',
#        'China TVBox', 'Gretel', 'Vertu', 'Evercoss', 'teXet', 'Qbex',
#        'Panasonic', 'Tagital', 'Dell', 'Maxvi']

device_brand преобразуем в три категории: Apple - 3, Samsung, Xiaomi, Huawei и Poco - 2, остальные бренды - 1
признак device_browser отбросим - 

In [ ]:
def foo_browser(x):
    t = 0
    if x in ['Chrome', 'Safari', 'YaBrowser']:
        t = 2
    elif x in ['Safari (in-app)', 'Android Webview', 'Samsung Internet']:
        t = 1
    else:
        t = 0
    return t

res['cat_browser'] = res['device_browser'].apply(lambda x: foo_browser(x))

In [6]:
res['device_brand'] = res['device_brand'].fillna('other')
res['device_brand'] = res['device_brand'].apply(lambda a: 1 if a != 'Apple' else 0)

In [7]:
columns_drop = ['device_model', 'device_os', 'utm_adcontent', 'device_browser', 
                'device_os', 'visit_date', 'visit_time', 'visit_number', 'client_id','session_id',
                'device_screen_resolution']
res = res.drop(columns_drop, axis = 1)
res = res.drop(['hit_page_path'], axis=1)
# res.head()

,utm_source,utm_medium,utm_campaign,utm_keyword,device_category,device_brand,geo_country,geo_city,target_event,month
0,0,banner,1,1,mobile,1,Russia,Zlatoust,0,11
1,1,cpm,1,1,mobile,1,Russia,Moscow,0,11
2,0,banner,1,1,mobile,1,Russia,Krasnoyarsk,0,12
4,0,cpc,0,0,mobile,1,Russia,Moscow,0,5
5,0,cpc,0,0,mobile,1,Russia,Moscow,0,5


In [ ]:
# df_plt = res.groupby(['month'])['target_event'].mean().to_frame().reset_index()
# df_plt.head()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.scatter(df_plt['month'], df_plt['target_event'])
# plt.title("Зависимость целевого действия от месяца")
# plt.xlabel('Месяц')
# plt.ylabel('Целевое действие')

# plt.show();

In [ ]:
# df_plt = res.groupby(['utm_source'])['target_event'].mean().to_frame().reset_index()

# plt.figure()
# plt.scatter(df_plt['utm_source'], df_plt['target_event'])
# plt.title("Зависимость целевого действия от использования соцсетей")
# plt.xlabel('Использование соцсетей')
# plt.ylabel('Целевое действие')

# plt.show();

In [ ]:
# df_plt = res.groupby(['utm_medium'])['target_event'].mean().to_frame().reset_index()
# df_plt.head()

In [ ]:
# plt.figure()
# plt.scatter(df_plt['utm_medium'], df_plt['target_event'])
# plt.title("Зависимость целевого действия от использования оргтрафика")
# plt.xlabel('Использование оргтрафика')
# plt.ylabel('Целевое действие')

# plt.show()

In [ ]:
# df_plt = res.groupby(['device_category'])['target_event'].mean().to_frame().reset_index()
# df_plt.head()

In [ ]:
# plt.figure()
# plt.scatter(df_plt['device_category'], df_plt['target_event'])
# plt.title("Зависимость целевого действия от типа устройства")
# plt.xlabel('Тип устройства')
# plt.ylabel('Целевое действие')

# plt.show()

In [ ]:
# df_plt = res.groupby(['device_brand'])['target_event'].mean().to_frame().reset_index()
# df_plt.head()

In [ ]:
# plt.figure()
# plt.scatter(df_plt['device_brand'], df_plt['target_event'])
# plt.title("Зависимость целевого действия от бренда")
# plt.xlabel('Бренд')
# plt.ylabel('Целевое действие')

# plt.show()

In [ ]:
# res = res.drop(['utm_campaign'], axis=1)
# res.head()

## GEO_*
### geo_country

In [ ]:
# res['is_rus'] = res['geo_country'].apply(lambda x: 1 if x=='Russia' else 0)
# res = res.drop(['geo_country'], axis=1)

### geo_city

In [ ]:
# df_plt = res.groupby(['geo_city'])['target_event'].mean().to_frame().reset_index()
# df_plt.sort_values(by=['target_event'], ascending=False).head()

In [ ]:
# df_plt[(df_plt['geo_city'] == 'Moscow') | (df_plt['geo_city'] == 'Saint Petersburg')]

In [ ]:
# view_occupancy(res)

In [8]:
res['geo_city'] = res['geo_city'].apply(lambda a: 1 if a in ['Moscow', 'Saint Petersburg'] else 0)

In [ ]:
# res = res.drop(['geo_city'], axis=1)
# res.head()

In [9]:
# res = res.sample(frac=1).reset_index(drop=True)
res = res.sample(n=200000).reset_index(drop=True)

cols_for_ohe = res.select_dtypes(include=['string', 'object', 'bool']).columns.tolist()
# print(cols_for_ohe)
# res.head()

['utm_medium', 'device_category', 'geo_country']


,utm_source,utm_medium,utm_campaign,utm_keyword,device_category,device_brand,geo_country,geo_city,target_event,month
0,0,banner,1,0,mobile,1,Russia,0,0,10
1,0,banner,1,1,mobile,1,Russia,1,0,12
2,0,banner,1,0,mobile,1,Russia,1,0,10
3,0,referral,1,0,mobile,0,Russia,1,0,9
4,0,cpc,1,1,mobile,0,Russia,1,0,11


In [10]:
ohe = OneHotEncoder(sparse=False, drop='first')

ohe.fit(res[cols_for_ohe])
transformed = ohe.transform(res[cols_for_ohe])
res[ohe.get_feature_names_out()] = transformed
df_ohed = res.drop(cols_for_ohe, axis=1)

%store df_ohed

Stored 'df_ohed' (DataFrame)
